In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


In [25]:
# Device Setting
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps')
device

device(type='mps')

In [26]:
# Seed setting
import random

def def_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def_seed(seed=123)

In [27]:
# Hyperparameters for Neural Network
epochs = 100
lr = 0.01
batch_size = 4096

In [30]:
# Data Load
df_train = pd.read_pickle('train.pkl')
df_test = pd.read_pickle('test.pkl')

In [39]:
df_train.Category.head(100).tolist()

['IAB22',
 'IAB22',
 'IAB22',
 nan,
 nan,
 'IAB22',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'IAB22',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'IAB9-30',
 'IAB3',
 'IAB9-30',
 nan,
 'IAB22',
 nan,
 'IAB9-30',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'IAB7-30%2CIAB6-7%2CIAB7-39%2CIAB7-45%2CIAB7-11',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'IAB22',
 nan,
 'IAB14',
 'IAB22',
 nan,
 'IAB13',
 nan,
 nan,
 nan,
 'IAB2',
 nan,
 'IAB9-30',
 nan,
 nan,
 'IAB22',
 'IAB22',
 nan,
 nan,
 'IAB22',
 'IAB2',
 nan,
 'IAB18-1',
 'IAB3',
 'IAB13',
 nan,
 nan,
 nan,
 'IAB2',
 'IAB9',
 nan,
 'IAB8',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'IAB22',
 nan,
 nan,
 nan,
 'IAB14',
 nan,
 'IAB13',
 'IAB14',
 nan,
 'IAB9-22%2CIAB9-12%2CIAB5-2%2CIAB19-24',
 nan,
 'IAB9-25',
 'IAB14',
 nan,
 'IAB22',
 'IAB3',
 'IAB13',
 'IAB18-1',
 'IAB22',
 'IAB13',
 nan]

## Dataset Setting

In [16]:
class MyDataset(Dataset):
    def __init__(self, df, if_test):
        self.df = df
        self.if_test = if_test
        if self.if_test:
            self.labels = self.df['winning'].values
            self.df = self.df.drop(columns=['winning']).values
        else:
            self.df = self.df.values

    def __getitem__(self, index):
        if self.if_test:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
    
    def __len__(self):
        return len(self.df)